In [35]:
import os, sys
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd

sys.path.append('/uio/kant/geo-ceed-u1/nilscp/Nils/Python/utils')
import raster

from shapely.geometry import Point

- selection crater
- get centroid
- get old coordinate systems
- project to new coordinate system (equirectangular based on lon, lat) (centroid), I guess I have to do that for 
- apply buffer to the centroid
- feature to envelope to polygon (make a square from the buffered geometry)
- reproject to old coordinate systems
- clip based on the bounding box / geometry
- convert to ascii (but maybe not necessary with this new code)

In [2]:
path = "/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2015_LARGE_CRATERS/layers"
filename = os.path.join(path, 'rayed_craters.shp')

In [3]:
# reading the shape file (craters)
df = gpd.read_file(filename)

In [45]:
# For SLDEM2015

path_raster = '/uio/kant/geo-ceed-u1/nilscp/Desktop/lunaspec/SLDEM2015'
in_raster = os.path.join(path_raster, 'Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif')
cliptype = 'geojson'

with rio.open(in_raster) as src:
    crs_wkt_src = src.crs.to_wkt()
    
for i in range(10,12):
    
    dict_crater = {'geometry': [df.geometry.loc[i]], 'index': [0]}
    dataframe_crater = pd.DataFrame(dict_crater)
    geodataframe_crater = gpd.GeoDataFrame(dataframe_crater, geometry=dataframe_crater.geometry)
    geodataframe_crater.crs = df.crs
    
    geom_geoseries = gpd.GeoSeries(geodataframe_crater.geometry)
    geom_geoseries.crs = df.crs
    
    centroids = geom_geoseries.centroid
    new_crs = df.crs.copy()
    new_crs['lat_ts'] = int(df.y_coord.loc[i])

    # get new coordinate system (this does not work yet)
    crs_wkt_dst = raster.crs_eqc(crs_wkt_src, df.y_coord.loc[i])
    
    # Change coordinates of centroids
    centroids_eq = centroids.to_crs(new_crs)
    
    # Does it make the buffer from the polygon? I do specifically 
    buffer_array = centroids_eq.buffer((df.Diam_km.loc[i]*0.5)*1500.0) # I will go for 0.75
    envelope_array = buffer_array.envelope
    envelope_global = envelope_array.to_crs(df.crs)
    
    tmp = envelope_global.__geo_interface__ 
    in_polygon = [tmp['features'][0]['geometry']]
    
    # clip
    fname_raster_clip = 'crater' + str(i).zfill(4) + '_clip.tif'
    clipped_raster = os.path.join(path_raster, fname_raster_clip)
    raster.clip_advanced(in_raster, in_polygon, cliptype, clipped_raster)
    
    # reproject
    fname_raster_reproj = 'crater' + str(i).zfill(4) + '.tif'
    reproj_raster = os.path.join(path_raster, fname_raster_reproj)
    
    # something to remember: the reprojection cause the raster to not have the resolution
    # there are two solutions: 1) work with rasters of different resolution or 2) downscale the raster
    # but you will miss some of the data...
    # doing 1) would be mean to change quite lot of the scripts previously written
    # but I don't think this is a problem for the mineralogical map.
    # the only question if I would have better detailed results with "native resolution"
    raster.reproject(clipped_raster, crs_wkt_dst, reproj_raster)   

In [34]:
# Olivine

path_raster = '/uio/kant/geo-ceed-u1/nilscp/Desktop/lunaspec/SLDEM2015'
in_raster = os.path.join(path_raster, 'Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif')
cliptype = 'geojson'

path_mineralogy = '/uio/kant/geo-ceed-u1/nilscp/Desktop/astra/Moon/Kaguya_mineralogy/'
in_raster_miner = os.path.join(path_mineralogy, 'Lunar_Kaguya_MIMap_MineralDeconv_OlivinePercent_50N50S_clip.tif')
paths = '/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2015_LARGE_CRATERS/ascii_olivine/'

with rio.open(in_raster) as src:
    crs_wkt_src = src.crs.to_wkt()
    
with rio.open(in_raster_miner) as src:
    crs_wkt_src_miner = src.crs.to_wkt()
    
# reproject main raster
#raster.reproject(in_raster_miner, crs_wkt_src, out_raster_miner)
#out_raster_miner = os.path.join(path_mineralogy, 'Lunar_Kaguya_MIMap_MineralDeconv_OlivinePercent_50N50S_reproj.tif')
   
for i in range(df.shape[0]):
    
    if ((df.loc[i,'y_coord'] >= 50.0) or (df.loc[i,'y_coord'] <= -50.0)):
        None
    else:   
        dict_crater = {'geometry': [df.geometry.loc[i]], 'index': [0]}
        dataframe_crater = pd.DataFrame(dict_crater)
        geodataframe_crater = gpd.GeoDataFrame(dataframe_crater, geometry=dataframe_crater.geometry)
        geodataframe_crater.crs = df.crs

        geom_geoseries = gpd.GeoSeries(geodataframe_crater.geometry)
        geom_geoseries.crs = df.crs

        centroids = geom_geoseries.centroid
        new_crs = df.crs.copy()
        new_crs['lat_ts'] = int(df.y_coord.loc[i])

        # get new coordinate system 
        crs_wkt_dst = raster.crs_eqc(crs_wkt_src, df.y_coord.loc[i])

        # Change coordinates of centroids
        centroids_eq = centroids.to_crs(new_crs)

        # Does it make the buffer from the polygon? I do specifically 
        buffer_array = centroids_eq.buffer((df.Diam_km.loc[i]*0.5)*1500.0) # I will go for 0.75
        envelope_array = buffer_array.envelope
        envelope_global = envelope_array.to_crs(df.crs) #df.crs

        tmp = envelope_global.__geo_interface__ 
        in_polygon = [tmp['features'][0]['geometry']]

        # clip
        fname_raster_clip = 'crater' + str(i).zfill(4) + '_clip.tif'
        clipped_raster = os.path.join(paths, fname_raster_clip)
        raster.clip_advanced(in_raster_miner, in_polygon, cliptype, clipped_raster)

        # reproject
        fname_raster_reproj = 'crater' + str(i).zfill(4) + '.tif'
        reproj_raster = os.path.join(paths, fname_raster_reproj)

        # something to remember: the reprojection cause the raster to not have the resolution
        # there are two solutions: 1) work with rasters of different resolution or 2) downscale the raster
        # but you will miss some of the data...
        # doing 1) would be mean to change quite lot of the scripts previously written
        # but I don't think this is a problem for the mineralogical map.
        # the only question if I would have better detailed results with "native resolution"
        raster.reproject(clipped_raster, crs_wkt_dst, reproj_raster)
               
        # delete clip file
        os.remove(clipped_raster)

In [26]:
## I will have to take into account the shifts for the coldspots (and only do it for the final, selected coldspots)

1471

In [24]:
df.loc[20,'y_coord']

56.8279360246

In [10]:
df.crs

{'proj': 'eqc',
 'lat_ts': 0,
 'lat_0': 0,
 'lon_0': 0,
 'x_0': 0,
 'y_0': 0,
 'a': 1737400,
 'b': 1737400,
 'units': 'm',
 'no_defs': True}

In [65]:
envelope_array.crs.values()

dict_values(['eqc', 13, 0, 0, 0, 0, 1737400, 1737400, 'm', True])

In [ ]:
envelope_global = envelope_array.to_crs(crs_wkt_src_miner) #df.crs

In [66]:
out_raster_miner = os.path.join(path_mineralogy, 'Lunar_Kaguya_MIMap_MineralDeconv_OlivinePercent_50N50S_reproj.tif')

In [ ]:
raster.reproject(in_raster_miner, crs_wkt_src, out_raster_miner)

In [30]:
filename_test = 'crater1021.tif'

path1 = '/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2015_LARGE_CRATERS/ascii_opx'
path2 = '/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2015_LARGE_CRATERS/ascii_plagio'

with rio.open(os.path.join(path1, filename_test)) as src:
    meta1 = src.meta
    
with rio.open(os.path.join(path2, filename_test)) as src:
    meta2 = src.meta

In [31]:
meta1

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -3.4028226550889045e+38,
 'width': 118,
 'height': 118,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["Equirectangular Moon",GEOGCS["GCS_Moon",DATUM["D_Moon",SPHEROID["Moon_localRadius",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",0],PARAMETER["standard_parallel_1",-2],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'),
 'transform': Affine(59.20725732087151, 0.0, -1236166.7625513263,
        0.0, -59.20725732087151, -67161.48316598707)}

In [32]:
meta2

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -3.4028226550889045e+38,
 'width': 118,
 'height': 118,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["Equirectangular Moon",GEOGCS["GCS_Moon",DATUM["D_Moon",SPHEROID["Moon_localRadius",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",0],PARAMETER["standard_parallel_1",-2],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'),
 'transform': Affine(59.20725732087151, 0.0, -1236166.7625513263,
        0.0, -59.20725732087151, -67161.48316598707)}